# [Computational Social Science] 
## 3-1 Classification - Student Version

In this lab we will cover **Classification** methods. Some of this might look familiar from your previous statistics courses where you fit models on binary or categorical outcomes.

---

## Data

We're going to use our [Census Income dataset](https://archive.ics.uci.edu/ml/datasets/Census+Income) dataset again for this lab. Load the dataset in, and explore it.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

%matplotlib inline
sns.set_style("darkgrid")

In [ ]:
# Create a list of column names, found in "adult.names"
col_names = ['age', 'workclass', 'fnlwgt',
            'education', 'education-num',
            'marital-status', 'occupation', 
             'relationship', 'race', 
             'sex', 'capital-gain',
            'capital-loss', 'hours-per-week',
            'native-country', 'income-bracket']

# Read table from the data folder
census = pd.read_table("../../data/adult.data", sep = ',', names = col_names)

In [ ]:
census.head()

Recall that before we try to train machine learning models on a dataset like this, we need to preprocess it. Preprocess the data to get it ready for training machine learning algorithms. Then, create a dataframe, **X**, that contains all of the features, and a series, **y**, that contains the target.

*Hint: Use `LabelBinarizer()` to transform your y variable, and `get_dummies()` to create dummy variables for your X's.* 

In [ ]:
# Target
...

# Features
...

### Class Balance

Before we start modeling, let's look at the distribution of the target variable. Visualize the distribution of the target variable ("income-bracket"). What do you notice? What do you think this pattern suggests about how easy or difficult it would be for a machine learning model to make the correct classifications?

In [ ]:
sns.displot(y)
plt.title("Distribution of Target Variable (Income Bracket)")
plt.xlabel('Income Bracket')
plt.ylabel('Count')
plt.show()

**Answer**:

### Data Splitting

Split the data into train, validation, and test sets. Take a look at some of the previous notebooks if you need a refresher!

In [ ]:
# Set seed
np.random.seed(10)

X_train, X_test, y_train, y_test = ...

X_train, X_validate, y_train, y_validate = ...

## Logistic Regression

Let's look at a [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). This example should look familiar from the Introduction to Machine Learning lab. Make a logistic regression model, fit it to the training data, and predict on the validation data.

In [ ]:
# create a model
logit_reg = ...

# fit the model
logit_model = ...

y_pred = ...

Next, we create a dataframe with the features and the logit coefficients (**Note**: For the logit coefficients we can use `np.transpose`, or extract the coefficients from the 1d array). 

Then, we plot the 10 coefficients with the largest absolute value.

In [ ]:
logit_data = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(logit_model.coef_))], axis = 1)
logit_data.columns = ['Feature', 'Coefficient']
logit_data['abs_coef'] = abs(logit_data['Coefficient'])

In [ ]:
sns.barplot(...)
plt.show()

We create a confusion matrix to visualize how well you did with your predictions. What do you notice? 

In [ ]:
cf_matrix = confusion_matrix(..., ..., normalize = "true")

df_cm = pd.DataFrame(cf_matrix, range(2), range(2))

df_cm = df_cm.rename(index=str, columns={0: "<=50k", 1: ">50k"})
df_cm.index = ["<=50k", ">50k"]
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

## Support Vector Machine

The next model we will look at is a [Support Vector Machine](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html). SVM is a non-parametric method that looks for the "best separating hyperplane" between two classes.

<img src="https://github.com/dlab-berkeley/Computational-Social-Science-Training-Program/blob/master/images/svm_kernel_machine.png?raw=true" style="width: 500px; height: 275px;" />

Initialize a Support Vector Machine model, fit it on the training data, and predict on the validation data. Visualize the confusion matrix. How does it compare to logistic regression?

In [ ]:
# create a model
svm = ...

# fit the model
svm_model = svm.fit(..., ...())

y_pred = svm_model.predict(...)

In [ ]:
cf_matrix = confusion_matrix(..., ..., normalize = "true")

df_cm = pd.DataFrame(cf_matrix, range(2), range(2))

df_cm = df_cm.rename(index=str, columns={0: "<=50k", 1: ">50k"})
df_cm.index = ["<=50k", ">50k"]
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

## Hyperparameter Tuning

As with sklearn's regression methods, we can also use [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to search for optimal hyperparameters. Choose one of the classification methods we have used so far and do a grid search to find the best hyperparameter values. Print out the optimal parameters, as well as the validation accuracy score from the best model (**Hint**: Take a look at the [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) method. **Note**: You might notice that the grid search takes a **very** long time to complete depending on the model and hyperparameters chosen.

In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore')
from sklearn.metrics import accuracy_score

param_grid = ...

logit_grid = GridSearchCV(..., param_grid, cv=3)
logit_grid.fit(..., ...)

best_index = np.argmax(logit_grid.cv_results_["mean_test_score"])
best_logit_pred = logit_grid.best_estimator_.predict(X_validate)

print(logit_grid.cv_results_["params"][best_index])
print('Validation Accuracy', accuracy_score(best_logit_pred, y_validate))

## Metrics

In machine learning, accuracy isn't the only metric that we might care about. Accuracy is an expression of ratio of correct observations relative to incorrect observations. This calculation alone does not tell us much about whether we did a good job predicting all of the various categories that we might be concerned about. Consider our census dataset. We saw earlier that the target data is not equally distributed - there were far more people with "<=50k" income. As we saw in our confusion matrices, our algorithms tended to predict observations belonging to the "<=50k" category remarkably well, but tended to do much worse with the ">50k" category. Why do you think this might be the case?

**Answer**: 

Let's define a few metrics that will help us move beyond accuracy as our only measure:

$$
True \space Positives = \sum({Predicted \space Positives = Observed \space Positives})
$$

$$
False \space Positives = \sum({Predicted \space Positives \space != Observed \space Positives})
$$

$$
True \space Negatives = \sum({Predicted \space Negatives = Observed \space Negatives})
$$

$$
False \space Negatives = \sum({Predicted \space Negatives \space != Observed \space Negatives})
$$

Imagine we were primarily interested in detecting whether someone is ">50k". We'll call this the "positive" class. A "predicted" observation is the value the model predicted, while the "observed" observation is the value in the ground-truth labels. So a "true positive" in this case would be instances when the model predicted someone to be in the ">50k" category AND they were in the ">50k" category in reality. Similarly, a false positive would be instances where the model predicted someone was in the ">50k" category when they were actually in the "<=50k" category in reality. Use your best model from hyperparameter to predict on the validation set and see how you did on each of these metrics. **Hint**: The confusion matrix is actually a great way to visualize all of these. What does each quadrant of the matrix correspond to in terms of these metrics?

In [ ]:
cf_matrix = confusion_matrix(..., ..., normalize = "true")

df_cm = pd.DataFrame(cf_matrix, range(2),
                  range(2))

df_cm = df_cm.rename(index=str, columns={0: "<=50k", 1: ">50k"})
df_cm.index = ["<=50k", ">50k"]
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, 
           annot=True,
           annot_kws={"size": 16},
           fmt='g')

plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

**Answer**: 

These metrics matter in the social sciences because we usually are not given balanced datasets, and we are oftentimes concerned with predicting rare events. Predicting rare events like fraud, credit defaults, and mortality is difficult. Optimizing on accuracy alone can be misleading if the algorithm just guesses the majority class every time without ever predicting the outcome of interest. 

## Accuracy

Recall the metrics we defined above: **True Positives (TP)**, **False Positives (FP)**, **True Negatives (TN)**, and **False Negatives (FN)**. Accuracy can be expressed as:

$$
Accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$

In plain language, what does this formula represent?

**Answer**: 

Write code to calculate the accuracy of your logistic regression. Calculate the number of true positives, false positives, true negatives, and false negatives and then calculate and print the accuracy.

*Tip: initialize the variables with 0 and use the `i` iterator in the for-loop when indexing your `y_validate` and `y_pred` variables.* 

In [ ]:
TP = ...
FP = ...
TN = ...
FN = ...

for i in range(len(y_pred)): 
    # True Positives. Hint, what two vectors need to equal each other and 1?
    if ... == ... == 1:
        TP += 1
    # False Positive. Hint, what vector needs to equal 1 while the other equals 0?
    if ... == 1 and ... != ...:
        FP += 1
    # True Negative
    if ... == ... ==0:
        TN += 1
    # False Negative
    if ... == 0 and y_pred[i]!= ...:
        FN += 1

In [ ]:
# Plug in the accuracy formula defined above!
accuracy = ...
print("Accuracy is", accuracy)

## Precision

Precision is a measure of how well calibrated predictions are. The formula for precision is:

$$
Precision = \frac{TP}{TP + FP}
$$

**Question**: In plain language, what does this formula tell us?

**Answer**:

Calculate and print the precision for the logistic regression.

In [ ]:
# Plug in the precision formula defined above!
precision = ...
print("Precision is", precision)

## Recall

Recall is defined as:

$$
Recall = \frac{TP}{TP + FN}
$$

**Question**: In plain language, what does the formula tell us?

**Answer**: 

Calculate the recall for our logistic regression model.

In [ ]:
# Plug in the recall formula defined above!
recall = ...
print("Recall is", recall)

**Question**: How did we do on precision and recall? Could you optimize for one or the other?

**Answer**: 

## F1 Score

The precision-recall tradeoff can be managed in a few different ways. One popular metric is the F1 score. It is defined as:

$$
F1 = 2 * \frac{precision * recall}{precision + recall}
$$

Calculate and print the f1 score.

In [ ]:
# Plug in the F1 formula defined above!
f1 = ...
print("F1 Score is", f1)

**Question**: How does F1 trade off between precision and recall? What are the advantages and disadvantages?

**Answer**: 

## AUC-ROC

[Area Under the Curve - Receiver Operating Characteristic (AUC-ROC)](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) is a popular method for seeing how well an algorithm does at separating between two classes. It is calculated by plotting the True Positive Rate against the False Positive Rate. Let's define these quantities:

$$
True \space Positive \space Rate(TPR) = Sensitivity = \frac{TP}{TP + FN}
$$

Hm, this formula looks familiar. In fact, it is exactly the same as Recall! Meanwhile, the False Positive Rate is:

$$
False \space Positive \space Rate (FPR) = 1 - Specificity = \frac{FP}{TN + FP}
$$

**Question**: Why does plotting TPR against FPR express separability between class labels?

**Answer**: 

Fill in the following code to plot the AUC-ROC for the logistic regression and a "no skill" model. Make sure to look up documentation as necessary.

In [ ]:
# roc curve and auc

# split into train/test sets
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(y_validate))]

# predict probabilities for logistic regression - enter your validation X!
lr_probs = logit_model.predict_proba(...)

# keep probabilities for the positive outcome only
lr_probs = ...

# calculate scores
ns_auc = roc_auc_score(...)
lr_auc = roc_auc_score(...)

# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(...)
lr_fpr, lr_tpr, _ = roc_curve(...)

# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

**Question**: How did the logistic regression do on the AUC-ROC metric? Compared to the "no skill" decision rule, was it a meaningful improvement?

**Answer**: 

## Over and Under Sampling

We have seen that imbalanced data can cause all sorts of problems and give us misleading results, especially if we only focus on accuracy. How can we correct for these problems? One simple method is to **resample** the data. For example, you might **oversample** the minority class or **undersample** the majority class. Let's use the [**imblearn**](https://imbalanced-learn.readthedocs.io/en/stable/api.html) to try this out. First, you might need to run the cell below to install the library. Anytime you use "!" in a Jupyter notebook, this will actually run a bash command.

In [ ]:
#!pip install imblearn

Now let's import the RandomOverSampler and RandomUnderSampler methods. 

**Question**: Why would we resample the training set, instead of the dataset or the validation/test sets?

**Answer**: 

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

Take a look at the first 15 values in `y_train` before we resample.

In [ ]:
...

Next use either the `RandomOverSampler` or `RandomUnderSampler` to resample the training set.
*Hint: we need to call the [`.fit_resample`](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.RandomOverSampler.html) method on our X_train and y_train variables.*

In [ ]:
random_over_sampler = RandomOverSampler(sampling_strategy=0.5)
random_under_sampler = RandomUnderSampler(sampling_strategy=0.5)

X_train_new, y_train_new = random_over_sampler...

Check the training labels again. Did anything change?

y_train_new[0:15]

**Question**: What do you notice about the resampled training targets? What might be some issues with over and undersampling?

**Answer**: 

Let's retrain the logistic regression model on the newly resampled data. How does AUC-ROC change?

In [ ]:
# fit the model
logit_model = ...

y_pred = ...

# roc curve and auc

# split into train/test sets
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(y_validate))]

# predict probabilities for logistic regression
lr_probs = logit_model.predict_proba(X_validate)

# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]

# calculate scores
ns_auc = roc_auc_score(y_validate, ns_probs)
lr_auc = roc_auc_score(y_validate, lr_probs)

# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))

# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_validate, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_validate, lr_probs)

# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

**Answer**: 

Overall, while sklearn puts together many of the methods we need to train, predict, and visualize the results of our machine learning, there are a lot of substantive choices involved. As you can see, even a slightly imbalanced dataset can cause problems. If you optimize only on accuracy, you might miss relevant aspects of the problem. Be mindful of the various metrics available, and decide which ones best answer the scientific question you have in mind.

---
Authored by Aniket Kesari. Updated by K. Quinn Fall 2021. 